In [1]:
import requests
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from datetime import datetime, timedelta
import yfinance as yf
# Set your API key
api_key = 'b07683d4fbed401490f767a345aebfdb'

# Define the date range
end_date = datetime.now()
start_date = end_date - timedelta(days=60)  # Extended to 60 days

# Fetch news articles in smaller chunks (e.g., daily) to ensure broader coverage
articles_list = []

for i in range(60):  # Adjusted for 60 days
    chunk_start_date = start_date + timedelta(days=i)
    chunk_end_date = chunk_start_date + timedelta(days=1)
    
    url = (f'https://newsapi.org/v2/everything?q=stock market&from={chunk_start_date.strftime("%Y-%m-%d")}'
           f'&to={chunk_end_date.strftime("%Y-%m-%d")}&language=en&sortBy=publishedAt&apiKey={api_key}')
    response = requests.get(url)
    
    if response.status_code == 200:
        articles = response.json().get('articles', [])
        articles_list.extend(articles)
    else:
        print(f"Error fetching data: {response.status_code}")

# Store the articles in a pandas df
df = pd.DataFrame(articles_list)
print(df[['publishedAt', 'title', 'description']])


Error fetching data: 426
Error fetching data: 426
Error fetching data: 426
Error fetching data: 426
Error fetching data: 426
Error fetching data: 426
Error fetching data: 426
Error fetching data: 426
Error fetching data: 426
Error fetching data: 426
Error fetching data: 426
Error fetching data: 426
Error fetching data: 426
Error fetching data: 426
Error fetching data: 426
Error fetching data: 426
Error fetching data: 426
Error fetching data: 426
Error fetching data: 426
Error fetching data: 426
Error fetching data: 426
Error fetching data: 426
Error fetching data: 426
Error fetching data: 426
Error fetching data: 426
Error fetching data: 426
Error fetching data: 426
Error fetching data: 426
Error fetching data: 426
Error fetching data: 426
               publishedAt                                              title  \
0     2024-05-27T23:46:48Z  US stock futures inch higher in slim trade as ...   
1     2024-05-27T23:46:06Z  Stocks splits are usually bullish. Here are 8 ...   
2     2

In [ ]:
# Phase 2: Pre-processing & Sentiment Analysis
nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()

# Function to get the sentiment score
def get_sentiment_score(text):
    return sid.polarity_scores(text)['compound'] if text else 0

df['sentiment'] = df['description'].apply(get_sentiment_score)

# Convert 'publishedAt' to datetime and ensure dates are parsed correctly
df['publishedAt'] = pd.to_datetime(df['publishedAt']).dt.date

# Group by day and calculate average sentiment
df['day'] = df['publishedAt']
daily_sentiment = df.groupby('day')['sentiment'].mean().reset_index()

# Group by week and calculate average sentiment
df['week'] = pd.to_datetime(df['day']).dt.to_period('W').apply(lambda r: r.start_time)
weekly_sentiment = df.groupby('week')['sentiment'].mean().reset_index()

# Display the daily and weekly sentiment
print(daily_sentiment.head(10))
print(weekly_sentiment.head(10))

# Display the range of dates
print(f"Date range: {df['day'].min()} to {df['day'].max()}")


In [ ]:
#print the first element of the df here the 1970 date
print(df.head(1))

In [ ]:
# Fetching stock market data for daily trends

# Define the stock ticker and date range
ticker = 'SPY'  # S&P 500 ETF as an example
end_date = datetime.now().date()
start_date = end_date - timedelta(days=60)  # Extended to 60 days
stock_data = yf.download(ticker, start=start_date, end=end_date, interval='1d')

# Calculate daily returns
stock_data['daily_return'] = stock_data['Adj Close'].pct_change()

# Reset the index to join with sentiment data
stock_data.reset_index(inplace=True)

# Convert Date to date
stock_data['Date'] = stock_data['Date'].dt.date

# Display the stock data
print("Stock Data with Daily Returns:\n", stock_data[['Date', 'daily_return']])


In [ ]:
# Verify the date ranges in daily_sentiment again after filtering
print("Filtered Daily Sentiment Dates:\n", daily_sentiment['day'].unique())

# Verify the date ranges in stock_data
print("Stock Data Dates:\n", stock_data['Date'].unique())

# Merging daily sentiment and stock data after filtering invalid dates
daily_combined_data = pd.merge(daily_sentiment, stock_data[['Date', 'daily_return']], left_on='day', right_on='Date', how='inner')

# Display the daily combined data
print("Daily Combined Data:\n", daily_combined_data.head(10))

# Handle missing data
daily_combined_data.dropna(subset=['daily_return'], inplace=True)

# Display the daily combined data after dropping NaNs
print("Daily Combined Data After Dropping NaNs:\n", daily_combined_data[['day', 'sentiment', 'daily_return']].head(10))

# Calculate daily correlation if there are enough data points
if len(daily_combined_data) > 1:
    daily_correlation = daily_combined_data['sentiment'].corr(daily_combined_data['daily_return'])
    print(f'Correlation between daily sentiment and daily stock performance: {daily_correlation}')
else:
    print("Not enough data points to calculate daily correlation.")
